# Quick Start - From Python

Here we present a broad overview of using the PyBigDFT library to drive BigDFT calculations using Python. Such an overview is intended to provide an initial walkthrough among the basic functionalities of the PyBigDFT objects and API.

The other notebooks of this section will present the main details of each of the functionalities described here.

## Installation

If you have installed from source, you should make sure you have setup the proper environment variables using the following command:
```
source install/bin/bigdftvars.sh
```

### Colab Installation

We will install BigDFT using Conda. First we need to setup conda.

In [ ]:
! pip install -q condacolab
import condacolab
condacolab.install()

When this finishes, the kernel will crash. This is necessary. After it restarts, you can proceed to install the BigDFT executable.

In [ ]:
! conda install -c "conda-forge/label/bigdft_dev" bigdft-suite  > /dev/null  2> /dev/null

And the required python libraries for this notebook.

In [ ]:
! pip install PyBigDFT  > /dev/null  2> /dev/null
! pip install py3dmol  > /dev/null  2> /dev/null

## System Manipulation
Here we define a system which is compsed of two fragments: H2 and Helium.

In [ ]:
from BigDFT.Systems import System
from BigDFT.Fragments import Fragment
from BigDFT.Atoms import Atom
from BigDFT.Visualization import InlineVisualizer

In [ ]:
# Create Three Atoms
at1 = Atom({"r": [0, 0, 0], "sym": "H", "units": "bohr"})
at2 = Atom({"r": [0, 0, 1.4], "sym": "H", "units": "bohr"})
at3 = Atom({"r": [10, 0, 0], "sym": "He", "units": "bohr"})

# Construct a System from Two Fragments (H2, He)
sys = System()
sys["H2:1"] = Fragment([at1, at2])
sys["He:2"] = Fragment([at3])

# Iterate Over The System
for fragid, frag in sys.items():
    for at in frag:
        print(fragid, at.sym, at.get_position())

In [ ]:
_ = sys.display()

## Calculation
Calculate the created system using a grid spacing of $0.4$ and the PBE functional. A logfile is generated from which we can access the computed properties. This logfile has built in properties and can be accessed like a dictionary.

In [ ]:
from BigDFT.Inputfiles import Inputfile
inp = Inputfile()
inp.set_hgrid(0.4)
inp.set_xc("PBE")
inp["perf"] = {"calculate_forces": False,
               "multipole_preserving": True}

inp

Here we can see that the input file takes the form of a python dictionary. We can now create a BigDFT calculator, which can be used to run BigDFT.

In [ ]:
from BigDFT.Calculators import SystemCalculator
calc = SystemCalculator(skip=False, verbose=False)

# as we're on colab, we don't want to use more than 1 core
# but if we were running on a larger computer, we could do e.g.
#calc = SystemCalculator(skip=False, verbose=False, omp=2, mpi_run='mpirun -np 2')

We can use this calculator to run BigDFT, but first let's run it with `dry_run=True`, which will create the input file without actually running BigDFT. Don't worry if it gives an error message.

In [ ]:
log = calc.run(sys=sys, input=inp, name="quick", run_dir="scratch", dry_run=True)

If we look at this input file, you can see it contains the parameters that we set, as well as the atomic positions. This is all the information needed to run this calculation, so you could e.g. copy the file to scarf and run there.

In [ ]:
!cat scratch/quick.yaml

Instead let's use the calculator to run BigDFT without the `dry_run` variable.

In [ ]:
log = calc.run(sys=sys, input=inp, name="quick", run_dir="scratch")

We can then extract some informationfrom the lofile, e.g. the memory or time taken.

In [ ]:
print(log.energy)
print(log.log["Memory Consumption Report"]
             ["Memory occupation"])
print(log.log["Timings for root process"]["Elapsed time (s)"])

## Periodic Systems (optional)
The code can treat also extended systems.
We setup a BCC unit cell of iron and perform the calculation using a 2x2x2 k-point grid with a Monkhorst-Pack grid. Since this calculation is more expensive, we'll also update the calculator to run the calculation in parallel.

In [ ]:
from BigDFT.UnitCells import UnitCell

In [ ]:
pat = Atom({"Fe": [0, 0, 0], "units": "angstroem"})
psys = System({"CEL:0": Fragment([pat])})
psys.cell = UnitCell([2.867, 2.867, 2.867], units="angstroem")

In [ ]:
_ = psys.display()

In [ ]:
inp = Inputfile()
inp.set_hgrid(0.4)
inp.set_xc("LDA")
inp.set_kpt_mesh(method='mpgrid',ngkpt=[2, 2, 2])

In [ ]:
calc = SystemCalculator(skip=True, verbose=False)

In [ ]:
log = calc.run(sys=psys, input=inp, name="psys", run_dir="scratch")

In [ ]:
_ = log.get_dos().plot()

## File I/O
Read and write a PDB file. The writing can be done with the `to_file` routine.

In [ ]:
sys.to_file('scratch/test.pdb')

For the reading, we can use the more pythonic way.
We here read another file.

In [ ]:
from BigDFT.IO import read_pdb
with open("scratch/test.pdb", "r") as ifile:
    sys = read_pdb(ifile)

And then display it.

In [ ]:
_ = sys.display()